In [1]:
%pip install av
%pip install moviepy

^C
Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/opt/conda/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/opt/conda/lib/python3.10/site-packages/pip/__main__.py", line 29, in <module>
    from pip._internal.cli.main import main as _main
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main.py", line 10, in <module>
    from pip._internal.cli.autocompletion import autocomplete
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/autocompletion.py", line 10, in <module>
    from pip._internal.cli.main_parser import create_main_parser
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/cli/main_parser.py", line 9, in <module>
    from pip._internal.build_env import get_runnable_pip
  File "/opt/conda/lib/python3.10/site-packages/pip/_internal/build_env.py", line 21, in <module>
    from pip.

In [64]:
import os
import torch
from torch.utils.data import Dataset
import numpy as np
from PIL import Image
import av
import random as rdm
import torchvision.transforms.v2 as transforms
from tqdm import tqdm
import datetime
import os
import torch
import torch.nn.functional as F

from torch.utils.data import Dataset
from torchvision.io import read_video
from torch.utils.data import DataLoader
import torchvision
import multiprocessing
import time
from torch.nn.utils.rnn import pad_sequence
from torchvision.datasets import UCF101

num_cores = multiprocessing.cpu_count()

print("This machine has {} CPU cores.".format(num_cores))

def set_seed(seed=42):
    rdm.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    return seed

set_seed()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

This machine has 8 CPU cores.
cpu


In [86]:
! wget https://github.com/rdfia/rdfia.github.io/raw/master/code/2-cd/utils.py
from utils import *

zsh:1: command not found: wget


In [65]:
# histogram with the distribution of the number of frames per video
import matplotlib.pyplot as plt
import numpy as np
from moviepy.editor import VideoFileClip

#file = "/kaggle/input/ucf101/UCF101/UCF-101/ApplyEyeMakeup/v_ApplyEyeMakeup_g01_c02.avi"
file = "/Users/in_vite/pCloud Drive/pCloud stockageUtile/Documents/DU MLIA/data/UCF101/SoccerPenalty/v_SoccerPenalty_g01_c04.avi"

clip = VideoFileClip(file)

print("FPS: ", clip.fps)
print("Height: ", clip.h)
print("Width: ", clip.w)
ratio_height_width = clip.h / clip.w
print("Ratio height/width: ", ratio_height_width)

FPS:  25.0
Height:  240
Width:  320
Ratio height/width:  0.75


In [88]:
class PermuteTensor(object):
    def __call__(self, img):
        return img.permute(1, 0, 2, 3)

def my_collate_fn(batch):
    # Check shape of each video tensor in the batch
    for item in batch:
        video, audio, label = item
        print('Video shape:', video.shape)

    # Now let's collate the batch as usual
    videos = torch.stack([item[0] for item in batch])
    labels = [item[1] for item in batch]
    return videos, labels
    
def get_ucf101(data_path, annotation_path, resized, num_frames=22, fold=1):
    col_jit = 0.2
    transform_outer = transforms.Compose([
        transforms.ColorJitter(brightness=col_jit, contrast=col_jit, saturation=col_jit, hue=col_jit),
        transforms.RandomHorizontalFlip(),
    ])

    transform_inner = transforms.Compose([
        PermuteTensor(),
        transforms.Resize(resized, antialias=True),
        #transforms.Lambda(resize_video),  # Resize videos
        transforms.ToImageTensor(),
        transforms.ConvertImageDtype(torch.float32),
        transforms.Normalize([0.5], [0.5]),
        #transforms.Lambda(pad_sequence)  # Custom padding
    ])

    transform_train = transforms.Compose([
        transform_inner,
        #transform_outer,
        #finalPermuteTensor()
    ])

    transform_test = transforms.Compose([
        transform_inner
    ])
    
    train_dataset = UCF101(data_path, annotation_path, num_frames, 1, fold=fold, transform=transform_train, output_format='TCHW')
    test_dataset = UCF101(data_path, annotation_path, num_frames, 1, fold=fold, train=False, transform=transform_test, output_format='TCHW')
    
    
    return train_dataset, test_dataset
    #return test_dataset

In [89]:
# Test the dataloader & save it in a file
#data_path = '/kaggle/input/ucf101/UCF101/UCF-101'
data_path = '/Users/in_vite/pCloud Drive/pCloud stockageUtile/Documents/DU MLIA/data/limitedUCF101'
#annotation_path = '/kaggle/input/ucf101/UCF101TrainTestSplits-RecognitionTask/ucfTrainTestlist'
annotation_path = '/Users/in_vite/pCloud Drive/pCloud stockageUtile/Documents/DU MLIA/data/ucfTrainTestList'

width = 110
height = int(width * ratio_height_width)
resize = (height, width)
frames = 16
fold = 1

train_set, valid_set = get_ucf101(data_path, annotation_path, resize, frames, fold)
#train_set = get_ucf101(data_path, annotation_path, resize, frames, fold)

100%|██████████| 43/43 [00:26<00:00,  1.61it/s]


In [90]:
for i, (videos, audio, labels) in enumerate(train_set):
    print('shape of batch :', videos.shape) # should get [batch_size, 3, seq_len, resized[0], resized[1]]
    print(labels) # should get [batch_size], the labels of the videos in the batch
    if i == 2:
        break
    
for i, (videos, audio, labels) in enumerate(valid_set):
    print('shape of batch :', videos.shape) # should get [batch_size, 3, seq_len, resized[0], resized[1]]
    print(labels) # should get [batch_size], the labels of the videos in the batch
    if i == 2:
        break
    
def my_collate_fn(batch):
    videos = torch.stack([item[0] for item in batch])
    labels = torch.Tensor([item[2] for item in batch]).long()  # Index at 2 instead of 1 because you also have 'audio' at index 1
    return videos, labels


train_loader = DataLoader(train_set, batch_size=4, shuffle=True, collate_fn=my_collate_fn)
valid_loader = DataLoader(valid_set, batch_size=4, shuffle=True, collate_fn=my_collate_fn)

for i, (videos, labels) in enumerate(train_loader):
    print('shape of batch :', videos.shape) # should get [batch_size, 3, seq_len, resized[0], resized[1]]
    print(labels) # should get [batch_size], the labels of the videos in the batch
    if i == 2:
        break
for i, (videos, labels) in enumerate(valid_loader):
    print('shape of batch :', videos.shape) # should get [batch_size, 3, seq_len, resized[0], resized[1]]
    print(labels) # should get [batch_size], the labels of the videos in the batch
    if i == 2:
        break

shape of batch : torch.Size([3, 16, 82, 110])
0
shape of batch : torch.Size([3, 16, 82, 110])
0
shape of batch : torch.Size([3, 16, 82, 110])
0
shape of batch : torch.Size([3, 16, 82, 110])
0
shape of batch : torch.Size([3, 16, 82, 110])
0
shape of batch : torch.Size([3, 16, 82, 110])
0
shape of batch : torch.Size([4, 3, 16, 82, 110])
tensor([0, 1, 1, 2])
shape of batch : torch.Size([4, 3, 16, 82, 110])
tensor([1, 1, 3, 1])
shape of batch : torch.Size([4, 3, 16, 82, 110])
tensor([1, 1, 1, 1])
shape of batch : torch.Size([4, 3, 16, 82, 110])
tensor([1, 2, 2, 2])
shape of batch : torch.Size([4, 3, 16, 82, 110])
tensor([3, 4, 1, 1])
shape of batch : torch.Size([4, 3, 16, 82, 110])
tensor([3, 2, 0, 4])


In [13]:
%mkdir prep_data_folder
torch.save(train_set, './prep_data_folder/trainset.pt')
torch.save(valid_set, './prep_data_folder/validset.pt')

mkdir: cannot create directory ‘prep_data_folder’: File exists


In [47]:
train_set = torch.load('./prep_data_folder/trainset.pt')
valid_set = torch.load('./prep_data_folder/validset.pt')
train_loader = DataLoader(train_set, batch_size=4, shuffle=True)
valid_loader = DataLoader(valid_set, batch_size=4, shuffle=False)
for i, (videos, audio, labels) in enumerate(train_loader):
    print('shape of batch :', videos.shape) # should get [batch_size, 3, seq_len, resized[0], resized[1]]
    print(labels) # should get [batch_size], the labels of the videos in the batch
    if i == 2:
        break
        
for i, (videos, audio, labels) in enumerate(valid_loader):
    print('shape of batch :', videos.shape) # should get [batch_size, 3, seq_len, resized[0], resized[1]]
    print(labels.shape) # should get [batch_size], the labels of the videos in the batch
    if i == 2:
        break
        
del train_set, valid_set

RuntimeError: stack expects each tensor to be equal size, but got [2, 57600] at entry 0 and [1, 0] at entry 1

In [82]:
import torch.nn as nn
    
class Simple3DCNN(nn.Module):
    def __init__(self, num_classes=6, in_channels=3, dropout_prob=0.5):
        super(Simple3DCNN, self).__init__()
        self.conv1 = nn.Conv3d(in_channels, 32, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn1 = nn.BatchNorm3d(32)
        self.relu = nn.ReLU(inplace=True) # ReLU activation
        self.maxpool1 = nn.MaxPool3d(kernel_size=(1, 2, 2), stride=(1, 2, 2))
        self.conv2 = nn.Conv3d(32, 64, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn2 = nn.BatchNorm3d(64)
        self.maxpool2 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.conv3 = nn.Conv3d(64, 128, kernel_size=(3, 3, 3), padding=(1, 1, 1))
        self.bn3 = nn.BatchNorm3d(128)
        self.maxpool3 = nn.MaxPool3d(kernel_size=(2, 2, 2), stride=(2, 2, 2))
        self.dropout = nn.Dropout(dropout_prob)
        self.fc1 = nn.Linear(3200, 2816//2)
        self.fc2 = nn.Linear(2816//2, num_classes)
        
    def forward(self, x):
        # print(x.shape)
        x = self.conv1(x)
        # print(x.shape)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool1(x)
        # print(x.shape)
        x = self.conv2(x)
        # print(x.shape)
        x = self.bn2(x)
        x = self.relu(x)
        x = self.maxpool2(x)
        # print(x.shape)
        x = self.conv3(x)
        # print(x.shape)
        x = self.bn3(x)
        x = self.relu(x)
        x = self.maxpool3(x)
        # print(x.shape)
        x = self.dropout(x)
        x = x.view(x.shape[0], -1)
        # print(x.shape)
        x = self.fc2(self.fc1(x))
        # print(x.shape)
        return x
    
    def save(self, file_name='model.pth'):
        model_folder_path = './model'
        if not os.path.exists(model_folder_path):
            os.makedirs(model_folder_path)
        file_name = os.path.join(model_folder_path, file_name)
        torch.save(self.state_dict(), file_name)
        

In [83]:
class Metric: 
    def __init__(self):
        self.loss_train = []
        self.loss_test = []
        self.acc_train = []
        self.acc_test = []
    
def epoch(data, model, criterion, optimizer=None, cuda=False):
    """
    Make a pass (called epoch in English) on the data `data` with the
     model `model`. Evaluates `criterion` as loss.
     If `optimizer` is given, perform a training epoch using
     the given optimizer, otherwise, perform an evaluation epoch (no backward)
     of the model.
    """

    # indicates whether the model is in eval or train mode (some layers behave differently in train and eval)
    model.eval() if optimizer is None else model.train()

    # objects to store metric averages
    avg_loss = AverageMeter()
    avg_top1_acc = AverageMeter()
    avg_top5_acc = AverageMeter()
    avg_batch_time = AverageMeter()
    global loss_plot

    # we iterate on the batches
    tic = time.time()
    for i, (input, target) in enumerate(tqdm(data)):
        
        if cuda: # only with GPU, and not with CPU
            input = input.cuda()
            target = target.cuda()

        # forward
        output = model(input)
        loss = criterion(output, target)

        # backward if we are training
        if optimizer:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # compute metrics
        prec1, prec5 = accuracy(output, target, topk=(1, 5))
        batch_time = time.time() - tic
        tic = time.time()

        # update
        avg_loss.update(loss.item())
        avg_top1_acc.update(prec1.item())
        avg_top5_acc.update(prec5.item())
        avg_batch_time.update(batch_time)
        if optimizer:
            loss_plot.update(avg_loss.val)
        # print info
        #if i % PRINT_INTERVAL == 0:
        #    print('[{0:s} Batch {1:03d}/{2:03d}]\t'
        #          'Time {batch_time.val:.3f}s ({batch_time.avg:.3f}s)\t'
        #          'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
        #          'Prec@1 {top1.val:5.1f} ({top1.avg:5.1f})\t'
        #          'Prec@5 {top5.val:5.1f} ({top5.avg:5.1f})'.format(
        #           "EVAL" if optimizer is None else "TRAIN", i, len(data), batch_time=avg_batch_time, loss=avg_loss,
        #           top1=avg_top1_acc, top5=avg_top5_acc))
            #if optimizer:
                #loss_plot.plot()

    # Print summary
    #print('\n===============> Total time {batch_time:d}s\t'
    #      'Avg loss {loss.avg:.4f}\t'
    #      'Avg Prec@1 {top1.avg:5.2f} %\t'
    #      'Avg Prec@5 {top5.avg:5.2f} %\n'.format(
    #       batch_time=int(avg_batch_time.sum), loss=avg_loss,
    #       top1=avg_top1_acc, top5=avg_top5_acc))

    return avg_top1_acc, avg_top5_acc, avg_loss

In [84]:
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn

def main(train, test, lr=0.1, epochs=5, cuda=False, dropout_prob=0.1, output_dir='./'):

    # define model, loss, optim
    model = Simple3DCNN(dropout_prob=dropout_prob)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.SGD(model.parameters(), lr)

    scheduler = lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

    if cuda: # only with GPU, and not with CPU
        cudnn.benchmark = True
        model = model.cuda()
        criterion = criterion.cuda()

    # init plots
    listm = []
    plot = AccLossPlot()
    global loss_plot
    loss_plot = TrainLossPlot()
    accs_train= []
    accs_test= []

    # We iterate on the epochs
    for i in range(epochs):
        m = Metric()

        # Train phase
        top1_acc, avg_top5_acc, loss = epoch(train, model, criterion, optimizer, cuda)
        # Update learning rate
        scheduler.step()

        # Test phase
        top1_acc_test, top5_acc_test, loss_test = epoch(test, model, criterion, cuda=cuda)
        # plot
        plot.update(loss.avg, loss_test.avg, top1_acc.avg, top1_acc_test.avg)
        m.acc_train = top1_acc.avg
        m.acc_test = top1_acc_test.avg
        m.loss_train = loss.avg
        m.loss_test = loss_test.avg
        listm.append(m)
        print( f"********** EPOCH {i+1} acc train={m.acc_train:.2f}%, acc test={m.acc_test:.2f}%, loss train={m.loss_train:.3f}, loss test={m.loss_test:.3f} **********")
        accs_train.append(top1_acc)
        accs_test.append(top1_acc_test)
    
    # save model
    time = datetime.datetime.now().strftime("%Y-%m-%d_%H:%M:%S")
    model_file_name = "model_" + time + ".pth"
    model.save(output_dir+model_file_name)
    
    return listm

In [91]:
from utils import *
#data_path = '/Users/in_vite/pCloud Drive/pCloud stockageUtile/Documents/DU MLIA/data/KTH'
#sequences_path = '/Users/in_vite/pCloud Drive/pCloud stockageUtile/Documents/DU MLIA/data/KTH/sequences.txt'
output_dir = '/kaggle/working/'
width = 12
height = int(width * ratio_height_width)
resized = (height, width)
batch_size = 2

lr = 0.0001
epochs = 2
cuda = torch.cuda.is_available()
num_workers = 0
dropout_prob = 0.1

num_frames = 100

# Get the data
#train, test, _ = get_loaders(data_path, sequences_path, resized, batch_size, num_frames)
#train = torch.load('/kaggle/working/prep_data_folder/trainset.pt')
#test = torch.load('/kaggle/working/prep_data_folder/validset.pt')
main(train_loader, valid_loader, lr, epochs, cuda, dropout_prob, output_dir)

  0%|          | 0/15013 [00:01<?, ?it/s]


RuntimeError: mat1 and mat2 shapes cannot be multiplied (4x66560 and 3200x1408)

<Figure size 640x480 with 0 Axes>

<Figure size 640x480 with 0 Axes>